In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

year1 = range(2001,2005) 
year2 = range(2006,2024)
years = list(year1) + list(year2)

scores_dict = {}
playoff_scores_dict = {}

In [15]:
for i in years:
    scores_dict[str(i)] = pd.read_csv('C:/Users/Gabriel/Documents/MIE368/Data/season_' + str(i) + '_data.csv')
    playoff_scores_dict['playoffs' + str(i)] = pd.read_csv('C:/Users/Gabriel/Documents/MIE368/Data/season_playoffs' + str(i) + '_data.csv')

## Game Result Elo Adjustment

In [14]:
Teams = scores_dict['2001']['Home'].unique().tolist()

team_changes = {
    'Atlanta Thrashers': 'Winnipeg Jets',
    'Phoenix Coyotes': 'Arizona Coyotes',
    'Mighty Ducks of Anaheim': 'Anaheim Ducks'
    # Add more mappings as needed
}

elo_ratings = {}

K = 6

KeyError: '2001'

In [ ]:
def calculate_prob_winning(home_team, away_team, home_goals, away_goals):
    
    Elo_diff_home = elo_ratings[home_team]-elo_ratings[away_team] + 50  #add extra for home-ice advantage   
    #Elo_diff_away = elo_ratings[away_team] - elo_ratings[home_team]
    prob_win_home = 1/(10**(-1*Elo_diff_home/400)+1)
    #prob_win_away = 1/(10**(-1*Elo_diff_away/400)+1)
    prob_win_away = 1 - prob_win_home
    
    return prob_win_home, prob_win_away, Elo_diff_home

In [ ]:
def margin_of_victory(home_goals, away_goals):
    
    mov = abs(home_goals - away_goals)
    mov_mult = 0.6686*np.log(mov)+0.8048
    
    return mov_mult

In [ ]:
# Function to update ELO ratings after a game
def update_elo(home_team, away_team, home_goals, away_goals, season):
    
    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)
    
    #print('Home team is ' + str(home_team))
    #print('Away team is ' + str(away_team))

    if home_team not in elo_ratings:
        # New teams added past a certain year start with a different ELO rating
        elo_ratings[home_team] = 1490 if season >= 2005 else 1380
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1490 if season >= 2005 else 1380

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, home_goals, away_goals)
    
     # Update Pregame Favorite Multiplier
    if home_goals > away_goals:
        home_win = 1
        #away_win = 0
        winner_elo_diff = Elo_diff_home
    elif home_goals < away_goals:
        home_win = 0
        #away_win = 1
        winner_elo_diff = -1*Elo_diff_home
    else:
        return
    
    pre_g_fav_h = home_win - home_prob  
    #pre_g_fav_a = away_win - away_prob  
    
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
     # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(home_goals, away_goals)
    #print('home goals:' + str(home_goals))
    #print('away goals:' + str(away_goals))
    #print('mult is: ' + str(mov_multiplier))
    
     # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
    #elo_shift_a = K * mov_multiplier * pre_g_fav_a
    #print('elo shift is: ' + str(elo_shift_h))
    
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h
    
    #print("Home team elo is: " + str(elo_ratings[home_team]))
    #print("Away team elo is: " + str(elo_ratings[away_team]))

### Testing Game Result Elo Adjustments

In [ ]:
'''
# Iterate over seasons and games
for season, df in scores_dict.items():
    for idx, row in df.iterrows():
        update_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], int(season))

truncated_dict = {k: round(v, 2) for k, v in elo_ratings.items()}
print(truncated_dict)
'''

In [ ]:
'''
# For testing on one season
for idx, row in scores_dict['2001'].iterrows():
    update_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], 2001)

print(elo_ratings)
'''

## End of Season Elo Adjustments

In [ ]:
# Initialize variables to store the final Elo ratings for the season
final_elo_ratings = {}
season_weight = 0.7
average_weight = 0.3

# Iterate over seasons and games
for season, df in scores_dict.items():
    for idx, row in df.iterrows():
        update_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], int(season))

    # Calculate the final Elo ratings for the season
    final_elo_ratings[season] = dict(elo_ratings)

    # Reset elo_ratings for the next season
    if int(season) < max(map(int, scores_dict.keys())):
        elo_ratings = {}

# Calculate the starting Elo ratings for the next season
for season in scores_dict.keys():
    if int(season) < max(map(int, scores_dict.keys())):
        next_season = str(int(season) + 1)
        for team in final_elo_ratings[season]:
            prev_season_elo = final_elo_ratings[season][team]
            league_avg_elo = sum(final_elo_ratings[season].values()) / len(final_elo_ratings[season])
            starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
            elo_ratings[team] = round(starting_elo, 2)

In [ ]:
elo_ratings

### Testing End of Season Adjustments

In [ ]:
test_seasons = ['2001', '2002']
test_dict = {}

final_elo_ratings = {}
season_weight = 0.7
average_weight = 0.3

for i in test_seasons:
    test_dict[i] = scores_dict[i]

#print(test_dict)

In [ ]:
# Iterate over seasons and games
for season, df in test_dict.items():
    for idx, row in df.iterrows():
        update_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], int(season))
    
    # Calculate the final Elo ratings for the season
    final_elo_ratings[season] = dict(elo_ratings)
    print('season: ' + str(season))
    print('End of season elo rating is: ' + str(final_elo_ratings))
    
    # Reset elo_ratings for the next season
    if int(season) < max(map(int, test_dict.keys())):
        elo_ratings = {}

# Calculate the starting Elo ratings for the next season
for season in test_dict.keys():
    if int(season) < max(map(int, test_dict.keys())):
        next_season = str(int(season) + 1)
        print('next season is: ' + str(next_season) + '\n')
        for team in final_elo_ratings[season]:
            prev_season_elo = final_elo_ratings[season][team]
            print('prev season is: ' + str(prev_season_elo))
            league_avg_elo = sum(final_elo_ratings[season].values()) / len(final_elo_ratings[season])
            print('league_avg: ' + str(league_avg_elo))
            starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
            print('starting_elo: ' + str(starting_elo))
            elo_ratings[team] = round(starting_elo, 2)

print(elo_ratings)